In [ ]:
!pip install pyforest

In [ ]:
import os
import pyforest

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data  = pd.read_csv("/kaggle/input/playstore-dataset/playstore-analysis.csv")

data.head(3)

In [ ]:
data.shape


In [ ]:
data.describe()

In [ ]:
data.columns

## DATA CLEANING

In [ ]:
# Drop specified columns from the DataFrame
data.drop(["App", "Current Ver", "Last Updated", "Current Ver", "Genres"], axis=1, inplace=True)

# Display information about the DataFrame
data.info()

In [ ]:
#Drop irrelevant columns from the dataset

data = data.drop_duplicates(keep = False)

#check length of the dataset to confirm duplicate rows have been deleted
d_l = len(data)
d_l

In [ ]:
data.isna().sum()

In [ ]:
#Category column cleaning steps
# 1. Find blanks

#data[data["Category"].isna()]

#Find the unique values in the column
#data["Category"].unique()
#Find the row with a value of 1.9 as category
data.loc[data["Category"] == "1.9"]

In [ ]:
#Drop the row

data.drop(index = 10472, inplace = True)


In [ ]:
#Ratings cleaning
#data["Rating"].unique()
print("Max value is: ", data["Rating"].max())
print("Min value is: ", data["Rating"].min())

The values fall within the acceptable range for ratings which is between 1 and 5

In [ ]:
#data["Rating"].isna().sum()- There are 1393 missing values in the column
#Replace missing values with mode
filler = data["Rating"].mode()
data["Rating"].fillna(filler [0], inplace = True)
data["Rating"].unique()

In [ ]:
#Reviews Cleaning 
data["Reviews"] = data["Reviews"].astype(int)
data.info()

In [ ]:
#Find null values in the column
data["Reviews"].isna().sum()

There are no null values in the column Reviews

In [ ]:
#Find blanks int the Size column
data["Size"].isna().sum()

There are no null values in the column Size

In [ ]:
# CLeaning the Installs Column
data["Installs"].isna().sum()

Theer are no null values in the column Installs

In [ ]:
# Remove the '+' at the end and convert to integer
data["Installs"] = pd.to_numeric(data["Installs"].str.replace(r'\D', '', regex=True))
data["Installs"].unique()

In [ ]:
# Cleaning the Type column
data[data["Type"].isna()]   #find the index of the row with the missing value
data.drop(index = 9148, inplace = True) #Drop the row 
data["Type"].unique()

In [ ]:
#Cleaning the Price Column
#Remove $ and convert to float
data["Price"] = pd.to_numeric(data["Price"].str.replace('$', ''))
data["Price"].unique()

In [ ]:
# #Cleaning the Android Version column
data[data["Android Ver"].isna()] #fnd indices for ros with missing values
indices = [4453, 4490] #pass the index values to a list
data.drop(indices, inplace = True)   # Drop the rows based on the index values
data["Android Ver"].unique()

In [ ]:
data["Android Ver"] = data["Android Ver"].str.replace('and up', '+')
data["Android Ver"] = data["Android Ver"].str.replace('W', '')
data["Android Ver"].unique()

In [ ]:
data["Android Ver"].mode()

In [ ]:
data.style.format({
    "Reviews": "{:,d}"
})
data.head()

In [ ]:
data.head()

## DATA ANALYSIS AND VISUALIZATION

In [ ]:
#Find number of apps on playstore by category
data.set_index('Category')
categories = data.groupby("Category")["Category"].count().sort_values(ascending = False).head(10)
categories

In [ ]:
categories = data["Category"].value_counts().tail(10).to_list()

categories

In [ ]:
#Find the categories with the most apps on playstore
top_10 = (data['Category'].value_counts()).iloc[:10]

#create bar chart to visualize them
top_10.plot(kind='bar', title ="No of Apps on Playstore by Category", figsize = [8,6])
plt.xlabel("Category")
plt.ylabel("No of Apps")
# plt.show()

The family category has the most apps on playstore followed by Games and Tools

In [ ]:
df_installs = pd.DataFrame(data.groupby("Category")["Installs"].sum().sort_values(ascending = False).head(10).astype(int))
df_installs.plot(kind = 'bar', figsize = [8,6], title = "Top 10 apps by Installs", color="Crimson")
plt.xlabel("Category")
plt.ylabel("No of Installs")
plt.show()

In [ ]:
df_installs